In [40]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd
data = pd.read_csv("engtomar.csv")
english_sentences = data["English"].tolist()
marathi_sentences = data["Marathi"].tolist()
marathi_sentences = [str(sentence) for sentence in marathi_sentences]
english_sentences = [str(sentence) for sentence in english_sentences]

In [41]:
tokenizer_eng = Tokenizer()
tokenizer_eng.fit_on_texts(english_sentences)
eng_seq = tokenizer_eng.texts_to_sequences(english_sentences)

tokenizer_ma = Tokenizer()
tokenizer_ma.fit_on_texts(marathi_sentences)
ma_seq = tokenizer_ma.texts_to_sequences(marathi_sentences)

vocab_size_eng = len(tokenizer_eng.word_index) + 1
vocab_size_ma = len(tokenizer_ma.word_index) + 1

# Padding
max_length = max(len(seq) for seq in eng_seq + ma_seq)
eng_seq_padded = pad_sequences(eng_seq, maxlen=max_length, padding='post')
ma_seq_padded = pad_sequences(ma_seq, maxlen=max_length, padding='post')

In [42]:
embedding_dim = 256
units = 512

# Encoder
encoder_inputs = Input(shape=(max_length,))
enc_emb = Embedding(input_dim=vocab_size_eng, output_dim=embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_length,))
dec_emb_layer = Embedding(input_dim=vocab_size_ma, output_dim=embedding_dim)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(vocab_size_ma, activation='softmax')
output = decoder_dense(decoder_outputs)

# Model
model = Model([encoder_inputs, decoder_inputs], output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])


In [43]:
X_train, X_val, y_train, y_val = train_test_split(eng_seq_padded, ma_seq_padded, test_size=0.2)
model.fit([X_train, X_train], y_train, validation_data=([X_val, X_val], y_val), epochs=50, batch_size=64)

Epoch 1/50
54/54 [==============================] - 12s 196ms/step - loss: 1.6277 - accuracy: 0.8820 - val_loss: 0.9445 - val_accuracy: 0.8969
Epoch 2/50
54/54 [==============================] - 10s 191ms/step - loss: 0.8451 - accuracy: 0.8979 - val_loss: 0.8728 - val_accuracy: 0.8975
Epoch 3/50
54/54 [==============================] - 10s 190ms/step - loss: 0.7733 - accuracy: 0.8987 - val_loss: 0.9029 - val_accuracy: 0.8954
Epoch 4/50
54/54 [==============================] - 11s 195ms/step - loss: 0.7578 - accuracy: 0.8983 - val_loss: 0.8689 - val_accuracy: 0.8984
Epoch 5/50
54/54 [==============================] - 10s 194ms/step - loss: 0.7171 - accuracy: 0.9008 - val_loss: 0.8801 - val_accuracy: 0.8989
Epoch 6/50
54/54 [==============================] - 11s 197ms/step - loss: 0.7008 - accuracy: 0.9011 - val_loss: 0.8807 - val_accuracy: 0.8991
Epoch 7/50
54/54 [==============================] - 10s 194ms/step - loss: 0.6842 - accuracy: 0.9015 - val_loss: 0.8867 - val_accuracy: 0.8992

In [44]:
model.save("translation_model.h5")


/Users/bhavishachaudhari/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
def translate_sentence(sentence):
    seq = tokenizer_eng.texts_to_sequences([sentence])
    padded = pad_sequences(seq, maxlen=max_length, padding='post')
    translated = np.argmax(model.predict([padded, padded]), axis=-1)
    
    translated_sentence = []
    for i in translated[0]:
        if i in tokenizer_ma.index_word:
            translated_sentence.append(tokenizer_ma.index_word[i])
        else:
            translated_sentence.append(' ') 
        
    return ' '.join(translated_sentence)
while True:
    input_sentence = input()
    translated_sentence = translate_sentence(input_sentence)
    print(f"Input: {input_sentence}")
    print(f"Translated: {translated_sentence}")

How are you
1/1 [==============================] - 0s 19ms/step
Input: How are you
Translated: तू कसा आहेस                              
I have lot of work
1/1 [==============================] - 0s 17ms/step
Input: I have lot of work
Translated: मला आज काम आहे                            
Let me go
1/1 [==============================] - 0s 21ms/step
Input: Let me go
Translated: मला जाऊ द्या                              
